## TFM: CLUSTERIZATION ALGORITHMS

In [ ]:
#Installs
!pip install xlrd
!pip install openpyxl
!pip install geneticalgorithm
!pip install pyeasyga

In [ ]:
#Imports
import numpy as np
import pandas as pd 
import math
#from datetime import datetime
#from contextlib import redirect_stdout #For the log files

import random

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
rc("text", usetex=False)


%matplotlib inline
import seaborn as sns

##Kmeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

##GMM
from sklearn.mixture import GaussianMixture

##DBSCAN
from sklearn.cluster import DBSCAN

##OPTICS
from sklearn.cluster import OPTICS
from sklearn.datasets import make_blobs

##Mean-Shift
from sklearn.cluster import MeanShift, estimate_bandwidth

##GA
from pyeasyga import pyeasyga
from scipy.stats import norm

##Likelihood 
from scipy.stats import norm
import scipy.stats as stats
import copy


In [ ]:
file_name = "../input/tfm-data/KBOXSales-DataTestCTES-25032021 2.xlsx"
df = pd.read_excel(file_name, engine='openpyxl', sheet_name='KMMVPRBQvm')
df.drop(['Unnamed: 92', 'Unnamed: 93',	'Unnamed: 94',	'Unnamed: 95',	'Unnamed: 96', 
         'Unnamed: 97',	'Unnamed: 98',	'Unnamed: 99'], axis = 1, inplace=True)

In [ ]:
df_s = df[:5495].copy() 
df_s.head()
#df.head() #5497

In [ ]:
train = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
              'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
              'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20']].copy()

train.dropna(axis=0, inplace=True)
train.reset_index(drop = True, inplace=True)
train

In [ ]:
#trains_cols = ['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', '%ContVta19', 'CtVta19', 
#'%ContMBto19', 'CtMBto19', 'EvoVta19', '%Dto19', '%DifDto19']
#test_cols = ['Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 
#'Key-WCI20', 'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20]

In [ ]:
#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
y = train['Key-P20'].copy()

labelEncoder = LabelEncoder()
labelEncoder.fit(y)
y = labelEncoder.transform(y)

## 0) Plots

### 0.1) Short codes, and key plots

In [ ]:
X_plot = train[['CodigoCLI', 'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20', 'KHR20', 
                'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20']].copy()

In [ ]:
#for i in X_plot:
#    if i == 'CodigoCLI':
#        continue
#    else:
#        plt.figure(figsize=(10,7))
#        sns.swarmplot(x=i, y=X_plot.index, data=X_plot, dodge=True, palette='viridis')
#        #sns.violinplot(x='Key-P20', y=X_plot.index, data=X_plot, split='True', palette='rainbow')
#        plt.title("Cluster distribution")

In [ ]:
#for i in X_plot:
#    if i == 'CodigoCLI':
#        continue
#    else:
#        plt.figure(figsize=(12,8))
#        sns.stripplot(x='Key-P20', y=X_plot.index, data=X_plot, jitter=True, dodge=True, palette='viridis')

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X_plot.iloc[:,0], X_plot.iloc[:,1]) #  X.iloc[:,2],  X.iloc[:,3]
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Data Distribution')
plt.show()
#X.drop('CodigoCLI', axis=1, inplace=True)

### 0.2) Indexes Plots

Always (y,x)

General (Value)
- Key-WCI - WCI/Ct2VM
- Key-R20 - WPI/IRP-Cli20base1

Sales

- Key-V20 - WSEI/IPEV20
- Key-P20 - SCI/CtVta20

Profit
- Key-R20 - WPI/IRP-Cli20base1
- Key-B20 - GMCI/CtMbto20

Quality
- Key-QV20 - TPI/IRT-Cli20Base1
- Key-QM20 - API/IRM-Cli20base1

--
- WCI = Ct2VM
- WPI = IRP-Clibase1
- WSEI = IPEV
- SCI = CtVta
- GMCI = CtMbto
- TPI = IRT-Clibase1
- API = IRM-Clibase1

In [ ]:
df_s.columns

In [ ]:
X_plot_pt1 = df_s[['Ct2VM20', 'Ct2VM', 
               'IRP-Cli20', #'evolIRP-Cli20',
               'IPEV20', 'CtVta20',
               'CtMBto19', 'CtMBto20', 
               'IRT-Cli20', #'evolIRT-Cli20', 
               'IRM-Cli20',]].copy() #'evolIRM-Cli20',

X_plot_pt2 = df_s[['IRP-Cli20base1',  'IRP-Cli20base1',  'IRP-Cli20base1']].copy()

X_plot_pt1.dropna(axis=0, inplace=True)
X_plot_pt2.dropna(axis=0, inplace=True)

X_plot_pt1.reset_index(drop = True, inplace=True)
X_plot_pt2.reset_index(drop = True, inplace=True)


In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_plot_pt1[['Ct2VM20', 'Ct2VM', 
        'IRP-Cli20', #'evolIRP-Cli20',
        'IPEV20', 'CtVta20',
        'CtMBto19', 'CtMBto20', 
        'IRT-Cli20', #'evolIRT-Cli20', 
        'IRM-Cli20',]] = scaler.fit_transform(X_plot_pt1)

X_plot_pt1

### 0.2) Indexes Plots

Always (y,x)

General (Value)
- Key-WCI - WCI/Ct2VM
- Key-R20 - WPI/IRP-Cli20base1

Sales

- Key-V20 - WSEI/IPEV20
- Key-P20 - SCI/CtVta20

Profit
- Key-R20 - WPI/IRP-Cli20base1
- Key-B20 - GMCI/CtMbto20

Quality
- Key-QV20 - TPI/IRT-Cli20Base1
- Key-QM20 - API/IRM-Cli20base1

--
- WCI = Ct2VM
- WPI = IRP-Clibase1
- WSEI = IPEV
- SCI = CtVta
- GMCI = CtMbto
- TPI = IRT-Clibase1
- API = IRM-Clibase1

In [ ]:
# Generate scatter plot
for i in range(len(X_plot_pt1.columns)):
    #colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', labels))
    plt.scatter(X_plot_pt1.index.values, X_plot_pt1.iloc[:,i], marker="o", picker=True)
    plt.title(f'Indicators clustering')
    plt.ylabel('Axis {}'.format(X_plot_pt1.iloc[:,i].name))
    plt.xlabel('Indices')
    plt.figtext(0.5, 0, "Avg:{}".format((X_plot_pt1.iloc[:,i]).mean()), ha="center", fontsize=7, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    plt.show()


In [ ]:
# Generate scatter plot
for i in range(len(X_plot_pt2.columns)):
    #colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', labels))
    plt.scatter(X_plot_pt2.index.values, X_plot_pt2.iloc[:,i], marker="o", picker=True)
    plt.title(f'Indicators clustering')
    plt.xlabel('Axis {}'.format(X_plot_pt2.iloc[:,i].name))
    plt.ylabel('Indices')
    plt.show()

## 1) Based on Partition
### 1.1) Kmeans

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X.iloc[:,0], X.iloc[:,1]) #  X.iloc[:,2],  X.iloc[:,3]
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Data Distribution')
plt.show()
#X.drop('CodigoCLI', axis=1, inplace=True)

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

#predictions from kmeans
pred = kmeans.predict(X)
frame = pd.DataFrame(X)
frame['cluster'] = pred
#frame.columns = ['Weight', 'Height', 'cluster']

In [ ]:
frame

In [ ]:
#plotting results
color=['blue','green','cyan', 'black']
for k in range(0,4):
    X = frame[frame["cluster"]==k]
    #plt.scatter(X["Weight"],X["Height"],c=color[k])
    plt.scatter(X.iloc[:,0], X.iloc[:,1], c=color[k])
    plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, init='random',tol=1e-04, random_state=0, algorithm = 'auto') 
kmeans.fit(X)

In [ ]:
# calculate distortion for a range of number of cluster
#distortions = []
#for i in range(1, 11):
#    km = KMeans(n_clusters=i, init='random',
#                n_init=10, max_iter=300,
#                tol=1e-04, random_state=0 )
#    km.fit(X)
#    distortions.append(km.inertia_)
#
## plot
#plt.plot(range(1, 11), distortions, marker='o')
#plt.xlabel('Number of clusters')
#plt.ylabel('Distortion')
#plt.show()

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X)):
        predict_me = np.array(X[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
    print(correct/len(X))

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
kmeans.fit(X_scaled)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X)):
        predict_me = np.array(X[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(correct/len(X))

### 1.1.1) Normalization, and Using less data / Using KBOX data

In this section we will first try and use less data, then we will use more initial data from Kbox

In [ ]:
#scaler = MinMaxScaler(feature_range=(-1, 1))
#X_norm =  scaler.fit_transform(X[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']])
#X_norm = pd.DataFrame(X_norm, columns=['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19'])

In [ ]:
## For general matrix WCI/IPoC, WPI/IRPo
##WCI - Vta19 y MBto19
##WPI - VtaTar19, VtaCos19, Rappel19

#X = train[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()

In [ ]:
#X = train[['Vta19', 'MBto19']].copy()  #WCI
X = train[['VtaTar19', 'VtaCos19', 'Rappel19']].copy() # WPI
y = train['Key-P20'].copy()

labelEncoder = LabelEncoder()
labelEncoder.fit(y)
y = labelEncoder.transform(y)

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

In [ ]:
kmeans = KMeans(n_clusters=4, init='random',tol=1e-04, random_state=0, algorithm = 'auto') 
kmeans.fit(X_scaled)

In [ ]:
X_scaled = np.array(X_scaled)
y = np.array(y)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(correct/len(X))

In [ ]:
## For sales matrix WSEI/IPEV, SCI/ICV
#WSEI- Vta19, 'MBto19'
#SCI - Vta19

X = train[['Vta19', 'MBto19']].copy()
y = train['Key-P20'].copy()

labelEncoder = LabelEncoder()
labelEncoder.fit(y)
y = labelEncoder.transform(y)

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, init='random',tol=1e-04, random_state=0, algorithm = 'auto') 
kmeans.fit(X_scaled)

X_scaled = np.array(X_scaled)
y = np.array(y)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(correct/len(X))

In [ ]:
## For profit matrix WPI/IRP, GMCI/ICMB
#WPI- 
#GMCI - 

X = train[['Vta19', 'MBto19']].copy()
y = train['Key-P20'].copy()

labelEncoder = LabelEncoder()
labelEncoder.fit(y)
y = labelEncoder.transform(y)

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, init='random',tol=1e-04, random_state=0, algorithm = 'auto') 
kmeans.fit(X_scaled)

X_scaled = np.array(X_scaled)
y = np.array(y)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(correct/len(X_scaled))

In [ ]:
#train.iloc[:,6:].head(5)
#KHR20 --  K Main Matrix
#KVP20 --  K Sales matrix
#KRB20 --  K Profit matrix
#KeyQVM20 --  K Sales Quality matrix

#Short-KHR20 --  Short K Main Matrix
#Short-KVP20 --  Short K Sales matrix
#Short-KRB20 --  Short K Profit matrix
#Short-KeyQVM20 -- Short K Sales Quality matrix

### 1.1.2) With KBOX data

In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x

In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

y = train_x['Key-P20'].copy()

labelEncoder = LabelEncoder()
labelEncoder.fit(y)
y = labelEncoder.transform(y)

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, init='random', tol=1e-04, random_state=0, algorithm = 'auto') 
kmeans.fit(X_scaled)

#X_scaled = np.array(X_scaled)
#y = np.array(y)

In [ ]:
correct_list = []

for i in range(15):
    y = train_x.iloc[:, 18+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
    correct_list.append(correct/len(X_scaled))
    print(correct/len(X_scaled))

In [ ]:
np.mean(correct_list)

In [ ]:
#calculate distortion for a range of number of cluster
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, init='random',
                n_init=10, max_iter=300,
                tol=1e-04, random_state=0 )
    km.fit(X_scaled)
    distortions.append(km.inertia_)

# plot
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
#plt.show()
plt.savefig('./elbow_plot.png', dpi=200)
plt.clf()


## 2) Based on Distribution
### 2.1) GMM 

In [ ]:
# training gaussian mixture model 
gmm = GaussianMixture(n_components=5, reg_covar=1e-4)
gmm.fit(X)

In [ ]:
#predictions from gmm
labels = gmm.predict(X)
frame = pd.DataFrame(X)
frame['cluster'] = labels
#frame.columns = ['Weight', 'Height', 'cluster']

In [ ]:
color=['blue','green','cyan', 'black']
for k in range(0,4):
    X = frame[frame["cluster"]==k]
    plt.scatter(X.iloc[:,1], X.iloc[:,2], c=color[k])
    #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
plt.show()

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
gmm.fit(X_scaled)

In [ ]:
for i in range(6):
    y = train.iloc[:, 5+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = gmm.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
    print(correct/len(X_scaled))

### 2.1.2) With KBOX data


In [ ]:
mu

mu_re = np.reshape(mu, (4, 17)) 

In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))

X_scaled = scaler.fit_transform(X)

# training gaussian mixture model 
gmm = GaussianMixture(n_components=3, reg_covar=1e-4, 
                      means_init = np.transpose(mu_means), 
                      weights_init = np.mean(pi_means, axis=0))
gmm.fit(X_scaled)

In [ ]:
no_init = []
init = []

for i in range(15):
    y = train_x.iloc[:, 18+i].copy()
    labelEncoder = LabelEncoder()
    labelEncoder.fit(y)
    y = labelEncoder.transform(y)
    y = np.array(y)
    
    correct = 0
    for i in range(len(X_scaled)):
        predict_me = np.array(X_scaled[i])
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = gmm.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1
            
    init.append(correct/len(X_scaled))
    print(correct/len(X_scaled))

In [ ]:
no_clusters = len(np.unique(labels))
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

### 2.1.3) Grid Search


In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)
# training gaussian mixture model 
gmm = GaussianMixture(n_components=3, reg_covar=1e-4)
gmm.fit(X_scaled)

In [ ]:
X_scaled

In [ ]:
color=['blue','green','cyan', 'black']
for k in range(0,4):
    X_scaled = frame[frame["cluster"]==k]
    plt.scatter(X_scaled.index.values, X_scaled.iloc[:,1],)
    #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
plt.show()

In [ ]:
num = 1

for i in range(7):
    reg = 1e-5 #* 100 * (y+1)

    for y in range(5):
        # training gaussian mixture model 
        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_scaled = scaler.fit_transform(X)
        #reg = 1e-5 * 100 * (y+1)
        gmm = GaussianMixture(n_components=i+1, reg_covar= reg)
        gmm.fit(X_scaled)
        
        #predictions from gmm
        labels = gmm.predict(X_scaled)
        frame = pd.DataFrame(X_scaled)
        frame['cluster'] = labels
        
        color=['blue','green','cyan', 'black']
        for k in range(0,4):
            X_scaled = frame[frame["cluster"]==k]
            plt.scatter(X_scaled.index.values, X_scaled.iloc[:,1],)
            #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
        # BOTTOM LABEL
        plt.figtext(0.5, 0, "Comps:{}, Reg_covar:{}".format(i+1, reg), ha="center", fontsize=7, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        plt.savefig('./gmm_film{}.png'.format(num))
        plt.clf()

        #plt.show()
        #plt.close()
        reg = reg * 10
        num = num + 1

## 3) Based on Density
### 3.1) DBSCAN

In [ ]:
#db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(X)
db = DBSCAN(eps=1, min_samples=50).fit(X)

labels = db.labels_
np.unique(labels)

### 3.1.2) With KBOX data


In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)


In [ ]:
db = DBSCAN(eps=0.1, min_samples=50).fit(X_scaled)

labels = db.labels_
np.unique(labels)

In [ ]:
no_clusters = len(np.unique(labels))
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

### 3.1.3) Grid Search


In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)

db = DBSCAN(eps=0.1, min_samples=50).fit(X_scaled)



In [ ]:
len(X_scaled)

In [ ]:
num = 1
epsilon = 0.1

for i in range(7):
    min_s = 50

    for y in range(5):
        # training gaussian mixture model 
        scaler = MinMaxScaler(feature_range=(-1, 1))
        X_scaled = scaler.fit_transform(X)
        labels = DBSCAN(eps = epsilon, min_samples = min_s).fit_predict(X_scaled)
        
        #predictions from gmm
        #labels = gmm.predict(X_scaled)
        frame = pd.DataFrame(X_scaled)
        frame['cluster'] = labels
        
        color=['blue','green','cyan', 'black']
        for k in range(0,4):
            X_scaled = frame[frame["cluster"]==k]
            plt.scatter(X_scaled.index.values, X_scaled.iloc[:,1],)
            #plt.scatter(X.iloc[:,2], X.iloc[:,3], c=color[k])
        # BOTTOM LABEL
        plt.figtext(0.5, 0, "Epsilon:{}, Min_samples:{}".format(epsilon, min_s), ha="center", fontsize=7, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        #plt.savefig('./dbscan_film{}.png'.format(num))
        #plt.show()

        #plt.clf()
        plt.show()
        #plt.close()
        min_s = min_s + 30
        
        num = num + 1
        print(min_s)
    epsilon = epsilon + 0.2
    print(epsilon)


### 3.2) OPTICS

In [ ]:
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19',]].copy() ##kbox data
             #'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             #'%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

In [ ]:
# Configuration options
epsilon = 2000.0
min_samples = 40
cluster_method = 'xi'
metric = 'minkowski'

# Compute OPTICS
db = OPTICS(max_eps=epsilon, min_samples=min_samples, cluster_method=cluster_method, metric=metric).fit(X)
labels = db.labels_

In [ ]:
lab = np.unique(labels)
lab

In [ ]:
no_clusters = len(np.unique(labels))
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

In [ ]:
# Generate scatter plot for training data
colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', labels))
plt.scatter(X.iloc[:,0], X.iloc[:,2], c=colors, marker="o", picker=True)
plt.title(f'OPTICS clustering')
plt.xlabel('Axis X[0]')
plt.ylabel('Axis X[1]')
plt.show()

In [ ]:
# Generate reachability plot
reachability = db.reachability_[db.ordering_]
plt.plot(reachability)
plt.title('Reachability plot')
plt.xlabel('Ordering of Points')
plt.ylabel('Reachability Distance')
plt.savefig('reachability.png', dpi=200)
#plt.show()


#ordering of the points as processed by OPTICS on the x-axis and the reachability distance on the y-axis. 

### 3.2.2) With KBOX data


In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X)


In [ ]:
# Configuration options
epsilon = 1000.0
min_samples = 40
cluster_method = 'xi'
metric = 'minkowski'

# Compute OPTICS
db = OPTICS(max_eps=epsilon, min_samples=min_samples, cluster_method=cluster_method, metric=metric).fit(X_scaled)
labels = db.labels_

In [ ]:
lab = np.unique(labels)
lab

In [ ]:
no_clusters = len(np.unique(labels))
no_noise = np.sum(np.array(labels) == -1, axis=0)

print('Estimated no. of clusters: %d' % no_clusters)
print('Estimated no. of noise points: %d' % no_noise)

In [ ]:
X_scaled_df = pd.DataFrame(X_scaled) 

In [ ]:
# Generate scatter plot for training data
colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', labels))
plt.scatter(X_scaled_df.iloc[:,0], X_scaled_df.iloc[:,2], c=colors, marker="o", picker=True)
plt.title(f'OPTICS clustering')
plt.xlabel('Axis X[0]')
plt.ylabel('Axis X[1]')
plt.show()

In [ ]:
# Generate reachability plot
reachability = db.reachability_[db.ordering_]
plt.plot(reachability)
plt.title('Reachability plot')
plt.show()

### 3.3) Mean-Shift

In [ ]:
# The following bandwidth can be automatically detected using
bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print("number of estimated clusters : %d" % n_clusters_)


### 3.3.2) With KBOX data


In [ ]:
X_scaled_df = pd.DataFrame(X_scaled) 

In [ ]:
# The following bandwidth can be automatically detected using
bandwidth = estimate_bandwidth(X_scaled_df, quantile=0.1, n_samples=100)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X_scaled_df)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
print("number of estimated clusters : %d" % n_clusters_)

In [ ]:
## Generate sample data
#centers = [[1, 1], [-1, -1], [1, -1]]
#X, _ = make_blobs(n_samples=10000, centers=centers, cluster_std=0.6)
#
## #############################################################################
## Compute clustering with MeanShift
#
## The following bandwidth can be automatically detected using
#bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=500)
#
#ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
#ms.fit(X)
#labels = ms.labels_
#cluster_centers = ms.cluster_centers_
#
#labels_unique = np.unique(labels)
#n_clusters_ = len(labels_unique)
#
#print("number of estimated clusters : %d" % n_clusters_)
#
## #############################################################################
## Plot result
#import matplotlib.pyplot as plt
#from itertools import cycle
#
#plt.figure(1)
#plt.clf()
#
#colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
#for k, col in zip(range(n_clusters_), colors):
#    my_members = labels == k
#    cluster_center = cluster_centers[k]
#    plt.plot(X[my_members, 0], X[my_members, 1], col + '.')
#    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
#             markeredgecolor='k', markersize=14)
#plt.title('Estimated number of clusters: %d' % n_clusters_)
#plt.show()

## 4) Normalization 

In [ ]:
X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
y = train['Key-P20'].copy()

#X.values.flatten()
#sum(n < 0 for n in X.values.flatten())

In [ ]:
X.min()

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
X_norm =  scaler.fit_transform(X[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']])
X_norm = pd.DataFrame(X_norm, columns=['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19'])

In [ ]:
X_norm['CodigoCLI'] = X['CodigoCLI']

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X_norm.iloc[:,2], X_norm.iloc[:,4])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Data Distribution')
plt.show()
#X_norm.drop('CodigoCLI', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X_norm.iloc[:,2], X_norm.iloc[:,3])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Data Distribution')
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(X_norm.iloc[:,2], X_norm.index,)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Data Distribution')
plt.show()

## 5) Initialization Genetic Algorithms

In [ ]:
data = '../input/tfm-data-2/bdims.csv'

In [ ]:
train_x = df_s[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
                '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
                'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
                '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',
                
                'Key-P20', 'Key-B20', 'Key-V20', 'Key-QV20', 'Key-QM20', 'Key-R20', 'Key-WCI20',
                'KHR20', 'KVP20', 'KRB20', 'KeyQVM20', 'Short-KHR20', 'Short-KVP20', 'Short-KRB20', 'Short-KeyQVM20',]].copy()

train_x.dropna(axis=0, inplace=True)
train_x.reset_index(drop = True, inplace=True)

#X = train[['CodigoCLI', 'Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19']].copy()
X = train_x[['Vta19', 'VtaTar19', 'VtaCos19', 'Rappel19', 'MBto19', 
             '%ContVta19', 'CtVta19', '%ContMBto19', 'CtMBto19', ##kbox data
             'PosAbcVta19', '%PosAbcVta19', 'AcuVta19', '%AcuVta19', 'PosAbcMBto',
             '%PosAbcMBto19', 'AcuMBto19', '%AcuMBto19',]].copy()

scaler = MinMaxScaler(feature_range=(100, 1000))
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = scaler.fit_transform(X.iloc[:,:2])
X_scaled

In [ ]:
#data = np.genfromtxt(data, delimiter=',', skip_header=1)
#data = data[:,-3] # select the "weight" column in the dataset

#data = np.array(X)

data = X_scaled
data = data[:,1]

N = data.shape[0] # number of data points
K = 6 # 4 components GMM
tot_iterations = 100 # stopping criterium

In [ ]:
# Step-1 (Init)
#mu = np.random.uniform(low=42.0, high=95.0, size=K) # mean
#sigma = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton

mu_high = np.mean(data) + np.std(data)
mu_low = np.mean(data) - np.std(data)

sigma_high = np.var(data) + np.std(data) #(np.var(data)/10/2/2) + (np.var(data)/10/2/2)/2 
sigma_low =  np.var(data) - np.std(data) #(np.var(data)/10/2/2) - (np.var(data)/10/2/2)/2

mu = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
sigma = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton

pi = np.ones(K) * (1.0/K) # mixing coefficients
r = np.zeros([K,N]) # responsibilities
nll_list = list() # used to store the neg log-likelihood (nll)

In [ ]:
for iteration in range(tot_iterations):
    #print(mu)
    
    # Step-2 (E-Step)
    for k in range(K):
        #print(norm.pdf(x=data, loc=mu[k], scale=sigma[k]))
        r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
    r = r / np.sum(r, axis=0) #[K,N] -> [N]
    #print(r)
        
    # Step-3 (M-Step)
    N_k = np.sum(r, axis=1) #[K,N] -> [K]
    #print('yo',N_k)
    for k in range(K):
        mu[k] = np.sum(r[k,:] * data) / N_k[k] # update mean
        #print(mu[k], N_k[k])
        numerator = r[k] * (data - mu[k])**2
        sigma[k] = np.sqrt(np.sum(numerator) / N_k[k]) # update std
    pi = N_k/N # update mixing coefficient
    #print(mu, numerator, sigma, pi)
    
    # Estimate likelihood and print info
    likelihood = 0.0
    for k in range(K):
        likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
    nll_list.append(-np.sum(np.log(likelihood)))
    #print("Iteration: "+str(iteration)+"; NLL: "+str(nll_list[-1]))
    #print("Mean "+str(mu)+"\nStd "+ str(sigma)+"\nWeights "+ str(pi)+"\n")
   
    # Step-4 (Check)
    if(iteration==tot_iterations-1): break

In [ ]:
# define and set function to create a candidate solution representation
def create_individual(data):
    individual = data[:]
    random.shuffle(individual)
    return individual


# define and set the GA's crossover operation
def crossover_function(parent_1, parent_2):
    #crossover_index = random.randrange(1, (len(parent_1)/2) )
    #child_1a = parent_1[:crossover_index]
    #child_1b = [i for i in parent_2 if i not in child_1a]
    #child_1 = child_1a + child_1b
    
    child_1a = random.sample(parent_1, 3) #6:3
    child_1b = [i for i in parent_2 if i not in child_1a]
    child_1b = random.sample(child_1b, 3) #3:1, 4:2, 5:3, 6:3
    child_1 = child_1a + child_1b
    
    #child_2a = parent_2[crossover_index:]
    #child_2b = [i for i in parent_1 if i not in child_2a]
    #child_2 = child_2a + child_2b
    
    child_2a = random.sample(parent_2, 3) #6:3
    child_2b = [i for i in parent_1 if i not in child_2a]
    child_2b = random.sample(child_2b, 3) #3:1, 4:2, 5:3, 6:3
    child_2 = child_2a + child_2b       

    return child_1, child_2


# define and set the GA's mutation operation
def mutate_function(individual):
    mutate_index1 = random.randrange(len(individual))
    mutate_index2 = random.randrange(len(individual))
    #print(mutate_index1, mutate_index2)
    individual[mutate_index1], individual[mutate_index2] = individual[mutate_index2], individual[mutate_index1]
    #return individual[mutate_index1], individual[mutate_index2]

# define and set the GA's selection operation
def selection(population):
    return random.choice(population)

def random_selection(population):
            """Select and return a random member of the population."""
            return self.random.choice(population)


In [ ]:
def mu_sigma_population_creator(data):
    mu_list_1 = []
    mu_list_2 = []
    
    sigma_list_1 = []
    sigma_list_2 = []
    
    mu_high = np.mean(data) + np.std(data)
    mu_low = np.mean(data) - np.std(data)

    sigma_high = np.var(data) + np.std(data)  
    sigma_low =  np.var(data) - np.std(data) 
  
    for i in range(25):        
        
        mu_1 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
        sigma_1 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
        #mu_1 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
        #sigma_1 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
        mu_list_1.append(list(mu_1))
        sigma_list_1.append(list(sigma_1))
    
    for y in range(25):
        mu_2 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
        sigma_2 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
        #mu_2 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
        #sigma_2 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
        mu_list_2.append(list(mu_2))
        sigma_list_2.append(list(sigma_2))
        
    return (mu_list_1, mu_list_2, sigma_list_1, sigma_list_2)

In [ ]:
def master_mu_sigma_population_creator(data):
    
    master_mu_list_1 = []
    master_mu_list_2 = []
    
    master_sigma_list_1 = []
    master_sigma_list_2 = []
    
    for i in range(17):
        #data = data[:,i]
    
        mu_list_1 = []
        mu_list_2 = []

        sigma_list_1 = []
        sigma_list_2 = []

        mu_high = np.mean(data[:,i]) + np.std(data[:,i])
        mu_low = np.mean(data[:,i]) - np.std(data[:,i])

        sigma_high = np.var(data[:,i]) + np.std(data[:,i])  
        sigma_low =  np.var(data[:,i]) - np.std(data[:,i]) 

        for i in range(25):

            mu_1 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
            sigma_1 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
            #mu_1 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
            #sigma_1 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
            mu_list_1.append(list(mu_1))
            sigma_list_1.append(list(sigma_1))
        
        master_mu_list_1.append(mu_list_1)
        master_sigma_list_1.append(sigma_list_1)

        for y in range(25):
            mu_2 = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
            sigma_2 = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
            #mu_2 = np.random.uniform(low=42.0, high=95.0, size=K) # mean
            #sigma_2 = np.random.uniform(low=5.0, high=10.0, size=K) # standard deviaiton
            mu_list_2.append(list(mu_2))
            sigma_list_2.append(list(sigma_2))
        
        master_mu_list_2.append(mu_list_2)
        master_sigma_list_2.append(sigma_list_2)


        
    return (master_mu_list_1, master_mu_list_2, master_sigma_list_1, master_sigma_list_2)

In [ ]:
population = mu_sigma_population_creator(data)
parent_1, parent_2 = population[0], population[1]
parent_3, parent_4 = population[2], population[3]

In [ ]:
population = master_mu_sigma_population_creator(data)
parent_1, parent_2 = population[0], population[1]
parent_3, parent_4 = population[2], population[3]

In [ ]:
def crossover_mutation(parent_1, parent_2, crossover_prob = 0.8, mutation_prob = 0.2):
    
    new_population = []
    population_size = 25
    child_1, child_2 = parent_1, parent_2
    
    for i in range(50):
        while len(new_population) < population_size:

            can_crossover = random.random() <= crossover_prob
            can_mutate = random.random() <= mutation_prob

            if can_crossover:
                child_1[i], child_2[i] = crossover_function(parent_1[i], parent_2[i])

            if can_mutate:
                mutate_function(child_1[i])
                mutate_function(child_2[i])

            new_population.append(child_1[i])
            if len(new_population) < population_size:
                new_population.append(child_2[i])
        
    return new_population

In [ ]:
def master_crossover_mutation(master_parent_1, master_parent_2, crossover_prob = 0.8, mutation_prob = 0.2):
    
    new_populations = []

    for i in range(17):
        
        new_population = []
        population_size = 25
        parent_1, parent_2 = master_parent_1[i], master_parent_2[i]
        child_1, child_2 = parent_1, parent_2

        for i in range(50):
            while len(new_population) < population_size:

                can_crossover = random.random() <= crossover_prob
                can_mutate = random.random() <= mutation_prob

                if can_crossover:
                    child_1[i], child_2[i] = crossover_function(parent_1[i], parent_2[i])

                if can_mutate:
                    mutate_function(child_1[i])
                    mutate_function(child_2[i])

                new_population.append(child_1[i])
                if len(new_population) < population_size:
                    new_population.append(child_2[i])
        
        new_populations.append(new_population)

    return new_populations

In [ ]:
new_population_mu = crossover_mutation(parent_1 = parent_1, parent_2 = parent_2)
new_population_sigma = crossover_mutation(parent_1 = parent_3, parent_2 = parent_4)

In [ ]:
new_population_mu = master_crossover_mutation(master_parent_1 = parent_1, master_parent_2 = parent_2)
new_population_sigma = master_crossover_mutation(master_parent_1 = parent_3, master_parent_2 = parent_4)

In [ ]:
def plot_distributions(data, data_sampled, mu, sigma, K, color="green", color_sampled="red", name='plot.png'):
    #matplotlib.rcParams['text.usetex'] = True
    #plt.rcParams.update({'font.size': 16})    
    data_sampled = np.clip(data_sampled, np.min(data), np.max(data))
    plt.hist(data, bins=15, color=color, alpha=0.45, density=True)
    plt.hist(data_sampled, bins=15, range=(np.min(data), np.max(data)), color=color_sampled, alpha=0.45, density=True)
    for k in range(K):
        curve = np.linspace(mu[k] - 10*sigma[k], mu[k] + 10*sigma[k], 100)
        color = np.random.rand(3)
        plt.plot(curve, stats.norm.pdf(curve, mu[k], sigma[k]), color=color, linestyle="--", linewidth=3)
    plt.ylabel(r"$p(x)$")
    plt.xlabel(r"$x$")
    plt.tight_layout()
    plt.xlim(20, 120)
    plt.savefig(name, dpi=200)
    plt.show()

def plot_likelihood(nll_list, num):
    #matplotlib.rcParams['text.usetex'] = True
    
    #plt.rcParams.update({'font.size': 16})
    plt.plot(np.arange(len(nll_list)), nll_list, color="black", linestyle="--", linewidth=3)
    plt.ylabel(r"(negative) log-likelihood")
    plt.xlabel(r"iteration")
    plt.tight_layout()
    plt.xlim(0, len(nll_list))
    #plt.savefig('nll1.png', dpi=200)
    plt.savefig('./real_data_{}.png'.format(num), dpi = 200)

    plt.show()

In [ ]:
def likelihood_calculator(dt, mu_list, sigma_list):    
    #data = np.genfromtxt(dt, delimiter=',', skip_header=1)#[:,-2]
    #data = data[:,-3]
    data = dt
    N = data.shape[0]
    K = 6 # two components GMM
    tot_iterations = 100 # stopping criteria

    # Step-1 (Init)
    #mu = np.random.uniform(low=42.0, high=95.0, size=K)
    #sigma = np.random.uniform(low=5.0, high=10.0, size=K)
    #pi = np.ones(K) * (1.0/K) # mixing coefficients
    #r = np.zeros([K,N]) # responsibilities
    #nll_list = list() # store the neg log-likelihood
    
    last_nll_list = list() ## Used to collect the last nulls for the mu
    complete_nll_list = list() ## Used to collect all the null lists
    
    #for 
    #mu_list_copy = mu_list.copy()
    #sigma_list_copy = sigma_list.copy()
    num = 1
    for mu_elem, sigma_elem in zip(mu_list, sigma_list):
        #print(mu_elem)
        #print(sigma_elem)
        
        #mu_high = np.mean(data) + np.std(data)
        #mu_low = np.mean(data) - np.std(data)
        
        #sigma_high = np.var(data) + np.std(data)  
        #sigma_low =  np.var(data) - np.std(data) 

        #mu = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
        #sigma = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton
    
        
        pi = np.ones(K) * (1.0/K) # mixing coefficients
        r = np.zeros([K,N]) # responsibilities
        nll_list = list() # store the neg log-likelihood
        
        mu = mu_elem
        sigma = sigma_elem
        
        for iteration in range(tot_iterations):  
            # Step-2 (E-Step)
            for k in range(K):   
                r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
            r = r / np.sum(r, axis=0) #[K,N] -> [N]

            # Step-3 (M-Step)
            N_k = np.sum(r, axis=1) #[K,N] -> [K]
            for k in range(K): 
                # update means
                mu[k] = np.sum(r[k,:] * data) / N_k[k]        
                # update variances
                numerator = r[k] * (data - mu[k])**2
                sigma[k] = np.sqrt(np.sum(numerator) / N_k[k])        
            # update weights
            pi = N_k/N 

            likelihood = 0.0

            for k in range(K):
                likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
            
            nll_list.append(-np.sum(np.log(likelihood)))        

        
            # Check for invalid negative log-likelihood (NLL)
            # The NLL is invalid if NLL_t-1 < NLL_t
            # Note that this can happen for round-off errors.
            if(len(nll_list)>=2):
                if(nll_list[-2]<nll_list[-1]): raise Exception("[ERROR] invalid NLL: "+str(nll_list[-2:]))

            #print("Iteration: " + str(iteration) + "; NLL: " + str(nll_list[-1]))
            #print("Mean " + str(mu) + "\nStd " + str(sigma) + "\nWeights " + str(pi) + "\n")     

            # Step-4 (Check)
            if(iteration==tot_iterations-1): 
                last_nll_list.append(nll_list[-1])
                break # check iteration
        print(str(nll_list[-1]))
        plot_likelihood(nll_list, num)
        num = num +1
    #data_gmm = sampler(pi, mu, sigma, N=1000)
    #plot_distributions(data, data_gmm, mu, sigma, K, color="green", color_sampled="red", name="plot_sampler.png")
    
    return last_nll_list   


In [ ]:
new_population_mu_copy = copy.deepcopy(new_population_mu)
new_population_sigma_copy = copy.deepcopy(new_population_sigma)

a = likelihood_calculator(data, new_population_mu_copy, new_population_sigma_copy)

In [ ]:
def master_likelihood_calculator(master_dt, master_mu_list, master_sigma_list):    
    #data = np.genfromtxt(dt, delimiter=',', skip_header=1)#[:,-2]
    #data = data[:,-
    master_final_mu = []
    master_final_sigma = []
    master_final_pi = []
    
    final_mu_list = []
    final_sigma_list = []
    final_pi_list = []
    
    for i in range(17):
        data = master_dt[:,i]
        N = data.shape[0]
        K = 5 # two components GMM
        tot_iterations = 100 # stopping criteria

        # Step-1 (Init)
        #mu = np.random.uniform(low=42.0, high=95.0, size=K)
        #sigma = np.random.uniform(low=5.0, high=10.0, size=K)
        #pi = np.ones(K) * (1.0/K) # mixing coefficients
        #r = np.zeros([K,N]) # responsibilities
        #nll_list = list() # store the neg log-likelihood

        last_nll_list = list() ## Used to collect the last nulls for the mu
        complete_nll_list = list() ## Used to collect all the null lists


        #mu_list_copy = mu_list.copy()
        #sigma_list_copy = sigma_list.copy()
        
        num = 1

        mu_list = master_mu_list[i]
        sigma_list = master_sigma_list[i]

        for mu_elem, sigma_elem in zip(mu_list, sigma_list):
            #print(mu_elem)
            #print(sigma_elem)

            pi = np.ones(K) * (1.0/K) # mixing coefficients
            r = np.zeros([K,N]) # responsibilities
            nll_list = list() # store the neg log-likelihood

            mu = mu_elem
            sigma = sigma_elem
            

            for iteration in range(tot_iterations):  
                # Step-2 (E-Step)
                for k in range(K):   
                    r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
                r = r / np.sum(r, axis=0) #[K,N] -> [N]

                # Step-3 (M-Step)
                N_k = np.sum(r, axis=1) #[K,N] -> [K]
                for k in range(K): 
                    # update means
                    mu[k] = np.sum(r[k,:] * data) / N_k[k]        
                    # update variances
                    numerator = r[k] * (data - mu[k])**2
                    sigma[k] = np.sqrt(np.sum(numerator) / N_k[k])        
                # update weights
                pi = N_k/N 

                likelihood = 0.0

                for k in range(K):
                    likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])

                nll_list.append(-np.sum(np.log(likelihood)))        


                # Check for invalid negative log-likelihood (NLL)
                # The NLL is invalid if NLL_t-1 < NLL_t
                # Note that this can happen for round-off errors.
                if(len(nll_list)>=2):
                    if(nll_list[-2]<nll_list[-1]): raise Exception("[ERROR] invalid NLL: "+str(nll_list[-2:]))

                #print("Iteration: " + str(iteration) + "; NLL: " + str(nll_list[-1]))
                #print("Mean " + str(mu) + "\nStd " + str(sigma) + "\nWeights " + str(pi) + "\n")     

                # Step-4 (Check)
                if(iteration == tot_iterations-1): 
                    last_nll_list.append(nll_list[-1])
                    break # check iteration
                    
            if min(mu) > 0: 
                final_mu_list.append(mu)
            if min(sigma) > 0: 
                final_sigma_list.append(sigma)
            if min(pi) > 0: 
                final_pi_list.append(pi)
        
            #plot_likelihood(nll_list, num)
            num = num +1
        #data_gmm = sampler(pi, mu, sigma, N=1000)
        #plot_distributions(data, data_gmm, mu, sigma, K, color="green", color_sampled="red", name="plot_sampler.png")
        master_final_mu.append(final_mu_list)
        master_final_sigma.append(final_sigma_list)
        master_final_pi.append(final_pi_list)
        
        
    #return last_nll_list   
    return (master_final_mu, master_final_sigma, master_final_pi)

In [ ]:
mu_means = []
sigma_means = []
pi_means = []

for i in range(17):
    mu_means.append(np.average(a[0][i], axis = 0))
    sigma_means.append(np.mean(a[1][i], axis = 0))
    pi_means.append(np.average(a[2][i], axis = 0))


In [ ]:
mu_means

In [ ]:
mu_means = []
sigma_means = []
pi_means = []

for i in range(17):
    mu_means.append(np.average(b_1[i], axis = 0))
    sigma_means.append(np.mean(b_2[i], axis = 0))
    pi_means.append(np.average(b_3[i], axis = 0))


In [ ]:
for i in range(17):
    b_1[i] = (list(filter(lambda x: x, b_1[i])))
    b_2[i] = (list(filter(lambda x: x, b_2[i])))
    b_3[i] = (list(filter(lambda x: x, b_3[i])))

In [ ]:
b_1 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in a[0] ]
b_2 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in a[1] ]
b_3 = [ [[x for x in y if not np.isnan(x)] for y in z] for z in a[2] ]


In [ ]:
new_population_mu_copy = copy.deepcopy(new_population_mu)
new_population_sigma_copy = copy.deepcopy(new_population_sigma)

a = master_likelihood_calculator(data, new_population_mu_copy, new_population_sigma_copy)

In [ ]:
len((list(filter(lambda x: x, a[0]))))
len((list(filter(lambda x: x, a[1]))))
len((list(filter(lambda x: x, a[2]))))

In [ ]:
def plot_distributions(data, data_sampled, mu, sigma, K, color="green", color_sampled="red", name='plot.png'):
    matplotlib.rcParams['text.usetex'] = True
    plt.rcParams.update({'font.size': 16})    
    data_sampled = np.clip(data_sampled, np.min(data), np.max(data))
    plt.hist(data, bins=15, color=color, alpha=0.45, density=True)
    plt.hist(data_sampled, bins=15, range=(np.min(data), np.max(data)), color=color_sampled, alpha=0.45, density=True)
    for k in range(K):
        curve = np.linspace(mu[k] - 10*sigma[k], mu[k] + 10*sigma[k], 100)
        color = np.random.rand(3)
        plt.plot(curve, stats.norm.pdf(curve, mu[k], sigma[k]), color=color, linestyle="--", linewidth=3)
    plt.ylabel(r"$p(x)$")
    plt.xlabel(r"$x$")
    plt.tight_layout()
    plt.xlim(20, 120)
    plt.savefig(name, dpi=200)
    plt.show()

def plot_likelihood(nll_list):
    #matplotlib.rcParams['text.usetex'] = True
    plt.rcParams.update({'font.size': 16})
    plt.plot(np.arange(len(nll_list)), nll_list, color="black", linestyle="--", linewidth=3)
    plt.ylabel(r"(negative) log-likelihood")
    plt.xlabel(r"iteration")
    plt.tight_layout()
    plt.xlim(0, len(nll_list))
    plt.savefig('nll.png', dpi=200)
    plt.show()
    
def sampler(pi, mu, sigma, N):
    data = list()
    for n in range(N):
        k = np.random.choice(len(pi), p=pi)
        sample = np.random.normal(loc=mu[k], scale=sigma[k])
        data.append(sample)
    return data

def main(dt):    
    #data = np.genfromtxt(dt, delimiter=',', skip_header=1)#[:,-2]
    #data = data[:,-3]
    
    N = dt.shape[0]
    K = 4 # two components GMM
    tot_iterations = 100 # stopping criteria

    # Step-1 (Init)
    mu_high = np.mean(data) + np.std(data)
    mu_low = np.mean(data) - np.std(data)
    
    sigma_high = np.var(data) + np.std(data)  
    sigma_low =  np.var(data) - np.std(data) 

    mu = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
    sigma = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton

    #mu = np.random.uniform(low=42.0, high=95.0, size=K)
    #sigma = np.random.uniform(low=5.0, high=10.0, size=K)
    
    pi = np.ones(K) * (1.0/K) # mixing coefficients
    r = np.zeros([K,N]) # responsibilities
    nll_list = list() # store the neg log-likelihood

    for iteration in range(tot_iterations):  
        # Step-2 (E-Step)
        for k in range(K):
            r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
        r = r / np.sum(r, axis=0) #[K,N] -> [N]
        
        # Step-3 (M-Step)
        N_k = np.sum(r, axis=1) #[K,N] -> [K]
        for k in range(K): 
            # update means
            mu[k] = np.sum(r[k,:] * data) / N_k[k]        
            # update variances
            numerator = r[k] * (data - mu[k])**2
            sigma[k] = np.sqrt(np.sum(numerator) / N_k[k])        
        # update weights
        pi = N_k/N 
        
        likelihood = 0.0
        for k in range(K):
            likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
        nll_list.append(-np.sum(np.log(likelihood)))        
        # Check for invalid negative log-likelihood (NLL)
        # The NLL is invalid if NLL_t-1 < NLL_t
        # Note that this can happen for round-off errors.
        if(len(nll_list)>=2):
            if(nll_list[-2]<nll_list[-1]): raise Exception("[ERROR] invalid NLL: "+str(nll_list[-2:]))
   
        print("Iteration: " + str(iteration) + "; NLL: " + str(nll_list[-1]))
        print("Mean " + str(mu) + "\nStd " + str(sigma) + "\nWeights " + str(pi) + "\n")     
   
        # Step-4 (Check)
        if(iteration==tot_iterations-1): break # check iteration
    
    return nll_list
    
    #plot_likelihood(nll_list)
    #data_gmm = sampler(pi, mu, sigma, N=1000)
    #plot_distributions(data, data_gmm, mu, sigma, K, color="green", color_sampled="red", name="plot_sampler.png")

In [ ]:
def main(dt):    
    #data = np.genfromtxt(dt, delimiter=',', skip_header=1)#[:,-2]
    #data = data[:,-3]
    
    mu_list = []
    sigma_list = []
    pi_list = []
    
    for i in range(17):
        
        data = dt[:,i]
        N = dt.shape[0]
        K = 4 # two components GMM
        tot_iterations = 100 # stopping criteria

        # Step-1 (Init)
        mu_high = np.mean(data) + np.std(data)
        mu_low = np.mean(data) - np.std(data)

        sigma_high = np.var(data) + np.std(data)  
        sigma_low =  np.var(data) - np.std(data) 

        mu = np.random.uniform(low = mu_low, high = mu_high, size=K) # mean
        sigma = np.random.uniform(low = sigma_low, high = sigma_high, size=K) # standard deviaiton

        #mu = np.random.uniform(low=42.0, high=95.0, size=K)
        #sigma = np.random.uniform(low=5.0, high=10.0, size=K)

        pi = np.ones(K) * (1.0/K) # mixing coefficients
        r = np.zeros([K,N]) # responsibilities
        nll_list = list() # store the neg log-likelihood

        for iteration in range(tot_iterations):  
            # Step-2 (E-Step)
            for k in range(K):
                r[k,:] = pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
            r = r / np.sum(r, axis=0) #[K,N] -> [N]

            # Step-3 (M-Step)
            N_k = np.sum(r, axis=1) #[K,N] -> [K]
            for k in range(K): 
                # update means
                mu[k] = np.sum(r[k,:] * data) / N_k[k]        
                # update variances
                numerator = r[k] * (data - mu[k])**2
                sigma[k] = np.sqrt(np.sum(numerator) / N_k[k])        
            # update weights
            pi = N_k/N 

            likelihood = 0.0
            for k in range(K):
                likelihood += pi[k] * norm.pdf(x=data, loc=mu[k], scale=sigma[k])
            nll_list.append(-np.sum(np.log(likelihood)))        
            # Check for invalid negative log-likelihood (NLL)
            # The NLL is invalid if NLL_t-1 < NLL_t
            # Note that this can happen for round-off errors.
            if(len(nll_list)>=2):
                if(nll_list[-2]<nll_list[-1]): raise Exception("[ERROR] invalid NLL: "+str(nll_list[-2:]))

            #print("Iteration: " + str(iteration) + "; NLL: " + str(nll_list[-1]))
            #print("Mean " + str(mu) + "\nStd " + str(sigma) + "\nWeights " + str(pi) + "\n")     

            # Step-4 (Check)
            if(iteration == tot_iterations-1): break # check iteration
        
        if min(mu) > 0: mu_list.append(mu)
        if min(sigma) > 0: sigma_list.append(sigma)
        if min(pi) > 0: pi_list.append(pi)
        
    return (mu_list, sigma_list, pi_list)
    
    #plot_likelihood(nll_list)
    #data_gmm = sampler(pi, mu, sigma, N=1000)
    #plot_distributions(data, data_gmm, mu, sigma, K, color="green", color_sampled="red", name="plot_sampler.png")

In [ ]:
a = main(data)

In [ ]:
len(a[0])

In [ ]:
def plot_likelihood(nll_list):
    #matplotlib.rcParams['text.usetex'] = True
    #plt.rcParams.update({'font.size': 16})
    
    plt.plot(np.arange(len(nll_list)), nll_list, color="black", linestyle="--", linewidth=3)
    plt.ylabel(r"(negative) log-likelihood")
    plt.xlabel(r"iteration")
    plt.tight_layout()
    plt.xlim(0, len(nll_list))
    plt.savefig('real_dataset_normal_ll.png', dpi=200)
    #plt.savefig('./gmm_film{}.png'.format(num))
    #plt.clf()

    plt.show()

In [ ]:
plot_likelihood(a)